##Unzip for the folder

In [ ]:
!unzip -q /content/rotten_tomato.zip -d /content/dataset_rotten_tomato/

##Code for **image processing**

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile

def preprocess_image(image):
    original = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.convertScaleAbs(original, alpha=1.3, beta=0)
    image = cv2.resize(image, (150, 150))
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    s_channel = hsv_image[:, :, 1]
    _, thresholded = cv2.threshold(s_channel, 50, 255, cv2.THRESH_BINARY)
    kernel = np.ones((5, 5), np.uint8)
    closed_mask = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)
    dilated_mask = cv2.dilate(closed_mask, kernel, iterations=2)
    segmented_image = cv2.bitwise_and(image, image, mask=dilated_mask)
    processed = segmented_image.astype('uint8')
    return processed

def save_processed_images(base_dir, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    file_paths = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_paths.append(os.path.join(root, file))
    for file_path in file_paths:
        image = cv2.imread(file_path)
        processed_image = preprocess_image(image)
        save_path = os.path.join(save_dir, os.path.basename(file_path))
        cv2.imwrite(save_path, cv2.cvtColor(processed_image, cv2.COLOR_RGB2BGR))

# Set the base directory to the location of your dataset and the directory to save processed images
base_dir = '/content/dataset_rotten_tomato'  # Update this path as needed
save_dir = '/content/processed_rotten_tomato_dataset'  # Update this path as needed
save_processed_images(base_dir, save_dir)

# Zip the processed images directory for easy download
zipf = zipfile.ZipFile('/content/processed_rotten_tomato_dataset.zip', 'w', zipfile.ZIP_DEFLATED)
for root, dirs, files in os.walk(save_dir):
    for file in files:
        zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(save_dir, '..')))
zipf.close()

# Provide a download link (specifically for Colab, adjust as necessary)
from google.colab import files
files.download('/content/processed_rotten_tomato_dataset.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>